In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.optimize import minimize
from scipy.stats import norm
import time
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [40]:
class BayesianOptimizer(object):
    def __init__(self, black_box_func, x_bound, num_ini, num_iter, kind, kappa, xi, printflag = True):
        self.eva_func = black_box_func # Black box function 
        self.x_bound = x_bound # Optimal domain
        self.num_iter = num_iter # Number of iterations
        self.num_ini = num_ini # Number of initial trial
        self.gp = GaussianProcessRegressor(random_state=1) # Gaussian process 
        self.acquisition = acquisition_func(kind, kappa, xi) # Acquisition function
        self.printflag = printflag
        
    def RunOptimizer(self):
        x, y = rand_evaluate(self.x_bound, self.eva_func, self.num_ini)
        self.gp.fit(x, y)
        x_max, y_max = x[y.argmax()], np.max(y)
        i = 0

        while i < self.num_iter:
            x_suggest = acq_max(x_max, x_bound, y_max, 10, self.acquisition, self.gp)
            y_suggest = self.eva_func([x_suggest])
            x = np.append(x,x_suggest.reshape(1,x.shape[1]),axis = 0)
            y = np.append(y, y_suggest, axis = 0)
            x_max, y_max = x[y.argmax()], np.max(y)
            self.gp.fit(x,y)
            i += 1
           
            if i%10 == 0 and self.printflag == True:
                print('%i th iteration right now! Current optimal value is %.6f'%(i, y_max))
        return x_max, y_max
    
def rand_evaluate(x_bound, black_box_func, num_step):
    # Generate num_step times uniformly distributed random number within the bound
    x_rand = np.random.uniform(x_bound[0], x_bound[1], size = (num_step, x_bound.shape[1]))
    
    # Evaluate the response at each generated x
    y_rand = black_box_func(x_rand)
    return x_rand, y_rand

def acq_max(x_max, x_bound, y_max, num_step, acquisition, gp):
    x_hist, y_hist = [], []
    x_tries = np.random.uniform(x_bound[0], x_bound[1], size = (num_step, x_bound.shape[1]))
    ys = acquisition.utility(x_tries, gp=gp, y_max=1)
    x_max = x_tries[ys.argmax()]
    max_acq = ys.max()
    x_seeds = np.random.uniform(x_bound[0], x_bound[1], size=(20, x_bound.shape[1]))

    for x_try in x_seeds:
        # Find the minimum of minus the acquisition function
        res = minimize(lambda x: -acquisition.utility(x.reshape(1, -1), gp = gp, y_max = y_max),
                       x_try.reshape(1, -1),
                       bounds=x_bound.T,
                       method="L-BFGS-B")

        # See if success
        if not res.success:
            continue

        # Store it if better than previous minimum(maximum).
        if max_acq is None or -res.fun[0] >= max_acq:
            x_max = res.x
            max_acq = -res.fun[0]

    # Clip output to make sure it lies within the bounds. Due to floating
    # point technicalities this is not always the case.
    return np.clip(x_max, x_bound[0], x_bound[1])

class acquisition_func(object):
    def __init__(self, kind, kappa, xi):

        self.kappa = kappa # If UCB is adopted, kappa is required
        self.xi = xi 
        self.kind = kind

    def utility(self, x, gp, y_max):
        if self.kind == 'ucb':
            return self._ucb(x, gp, self.kappa)
        if self.kind == 'ei':
            return self._ei(x, gp, y_max, self.xi)
        if self.kind == 'poi':
            return self._poi(x, gp, y_max, self.xi)

    @staticmethod
    def _ucb(x, gp, kappa):
        mean, std = gp.predict(x, return_std=True)
        std = std.reshape([1,-1])
        return mean + kappa * std

    @staticmethod
    def _ei(x, gp, y_max, xi):
        mean, std = gp.predict(x, return_std=True)
        std = std.reshape([1,-1])
        z = (mean - y_max - xi)/std
        return (mean - y_max - xi) * norm.cdf(z) + std * norm.pdf(z)

    @staticmethod
    def _poi(x, gp, y_max, xi):
        mean, std = gp.predict(x, return_std=True)
        std = std.reshape([1,-1])
        z = (mean - y_max - xi)/std
        return norm.cdf(z)

In [37]:
def test_func(x):
    for temp in x:
        y = [-(temp**2).sum() for temp in x]
    return np.array(y)

In [42]:
x_bound = np.array([[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],[1,1,1,1,1,1,1,1,1,1]])
trail = BayesianOptimizer(test_func, x_bound, 2, 600, 'ucb', 0.01, 0.1, True)
trail.RunOptimizer()

10 th iteration right now! Current optimal value is -1.767224
20 th iteration right now! Current optimal value is -1.767224
30 th iteration right now! Current optimal value is -1.767224
40 th iteration right now! Current optimal value is -1.767224
50 th iteration right now! Current optimal value is -1.767224
60 th iteration right now! Current optimal value is -1.767224
70 th iteration right now! Current optimal value is -1.767224
80 th iteration right now! Current optimal value is -1.767224
90 th iteration right now! Current optimal value is -1.767224
100 th iteration right now! Current optimal value is -1.767224
110 th iteration right now! Current optimal value is -1.767224
120 th iteration right now! Current optimal value is -1.767224
130 th iteration right now! Current optimal value is -1.767224
140 th iteration right now! Current optimal value is -1.250548
150 th iteration right now! Current optimal value is -0.718430
160 th iteration right now! Current optimal value is -0.418646
1

(array([ 0.00038568, -0.00091107,  0.00044543, -0.00017682,  0.00059072,
        -0.00054858,  0.00051628,  0.00028755, -0.00033142,  0.00056277]),
 -2.634148594161471e-06)